In [1]:
import numpy as np

fileInput = open('data.txt', mode='r')
dataInput = [cubeLine.strip() for cubeLine in fileInput.readlines()]
dataInput = np.array([[c for c in cubeLine] for cubeLine in dataInput ])
fileInput.close()

In [2]:
def prepareVolumeBeforeIter(volume):
    newVolume=[]
    nbLine = volume[0].shape[0]; nbCol = volume[0].shape[1]

    for layout in volume:
        layout = [(nbCol+2)*['.']] + [['.']+line+['.'] for line in layout.tolist()] + [(nbCol+2)*['.']] 
        newVolume.append(np.array(layout))
    newVolume.append(np.array([['.' for j in range(nbCol+2)] for i in range(nbLine+2)]))
    return newVolume

def findNeighborsInLayout(x,y,layout, sameLayout=False):
    nbLine = layout.shape[0]; nbCol = layout.shape[1]
    directions = [(1,0),(-1,0),(0,1),(0,-1),(-1,-1),(1,1),(-1,1),(1,-1)]
    allDire = directions if sameLayout else directions + [(0,0)]
    return [layout[x+i,y+j] for i,j in allDire if 0<=x+i<nbLine and 0<=y+j<nbCol]

def findNeighborsPart1(x,y,z, volume):
    zMax = len(volume)
    neigb = []
    for i in [-1,0,1]:
        currentZ = z+i
        if(abs(currentZ) < zMax):
            neigb += findNeighborsInLayout(x,y,volume[abs(currentZ)], i==0)
    return neigb

def changeValuePart1(x, y, z, volume):
    if(volume[z][x,y] == '#'):
        return '#' if findNeighborsPart1(x, y, z, volume).count('#') in [2,3] else '.'
    if(volume[z][x,y] == '.'):
        return '#' if findNeighborsPart1(x, y, z, volume).count('#') == 3 else '.'

def simulateOneCyclePart1(initialVolume):
    preVolume = prepareVolumeBeforeIter(volume=initialVolume)
    newVolume = []
    for z in range(len(preVolume)):
        layoutZ = np.array([[changeValuePart1(x,y,z, preVolume) for y in range(preVolume[z].shape[1])] for x in range(preVolume[z].shape[0])])
        newVolume.append(layoutZ)
    return newVolume

def simulateNCyclePart1(initialVolume, n):
    volume = initialVolume
    for i in range(n):
        volume = simulateOneCyclePart1(volume)
    return volume

In [3]:
def countActiveCube1(volume):
    s = 0
    for i in range(len(volume)):
        activeCubes = sum(volume[i][j].tolist().count('#') for j in range(volume[i].shape[0]))
        s+= activeCubes if i==0 else 2*activeCubes
    return s

In [4]:
def convertInputToNewHyperV(inputValue):
    hV = [[[]]]
    for x in range(inputValue.shape[0]):
        line = []
        for y in range(inputValue.shape[1]):
            line.append(inputValue[x,y])
        hV[0][0].append(line)
    return hV

In [5]:
def getNeighborsPart2(Ow,Oz,Ox,Oy, hyperV):
    wLen, zLen, xLen, yLen  = len(hyperV), len(hyperV[0]), len(hyperV[0][0]), len(hyperV[0][0][0])
    neighbors=[]

    for w in [-1,0,1]:
        for z in [-1,0,1]:
            for x in [-1,0,1]:
                for y in [-1,0,1]:
                    if(not(w==0 and z==0 and x==0 and y==0)):
                        if (0<=abs(Ow+w)<wLen and 0<=abs(Oz+z)<zLen 
                        and 0<=Ox+x<xLen and 0<=Oy+y<yLen):
                            neighbors.append(hyperV[abs(Ow+w)][abs(Oz+z)][Ox+x][Oy+y])
    return neighbors
                    

In [6]:
def prepareVolumeBeforeIter2(volume):
    newVolume, nbLine, nbCol = [], len(volume[0]), len(volume[0][0])

    for layout in volume:
        layout = [(nbCol+2)*['.']] + [['.']+line+['.'] for line in layout] + [(nbCol+2)*['.']] 
        newVolume.append(layout)
    newVolume.append([['.' for j in range(nbCol+2)] for i in range(nbLine+2)])
    return newVolume


def prepareHyperV(hyperV):
    nbLayout,nbLine,nbCol = len(hyperV[0]),len(hyperV[0][0]),len(hyperV[0][0][0])
    
    newHyperV = []
    for volume in hyperV:
        volume = prepareVolumeBeforeIter2(volume)
        newHyperV.append(volume)
    newHyperV.append([ [['.' for j in range(nbCol+2)] for i in range(nbLine+2)] for k in range(nbLayout+1)])
    return newHyperV

In [7]:
def changeValuePart2(x, y, z, w, hyperV):
    if(hyperV[w][z][x][y] == '#'):
        return '#' if getNeighborsPart2(w, z, x, y, hyperV).count('#') in [2,3] else '.'
    if(hyperV[w][z][x][y] == '.'):
        return '#' if getNeighborsPart2(w, z, x, y, hyperV).count('#') == 3 else '.'

def simulateOneCyclePart2(initialHyperV):
    preHyperV = prepareHyperV(initialHyperV)
    newHyperV = []
    
    for w in range(len(preHyperV)):
        volume = []
        for z in range(len(preHyperV[0])):
            layout = []
            for x in range(len(preHyperV[0][0])):
                ax = []
                for y in range(len(preHyperV[0][0][0])):
                    ax.append(changeValuePart2(x,y,z,w, preHyperV))
                layout.append(ax)
            volume.append(layout)
        newHyperV.append(volume)
    return newHyperV

def simulateNCyclePart2(initialHyperV, n):
    hyperV = initialHyperV
    for i in range(n):
        hyperV = simulateOneCyclePart2(hyperV)
    return hyperV

In [8]:
def countActiveCube2(hyperV):
    s = 0
    for w in range(len(hyperV)):
        sw = 0
        for z in range(len(hyperV[0])):
            activeCubes = sum(hyperV[w][z][x].count('#') for x in range(len(hyperV[0][0])))
            sw+= activeCubes if z==0 else 2*activeCubes
        s+= sw if w==0 else 2*sw
    return s

In [9]:
print('Part 1 :', countActiveCube1(simulateNCyclePart1([dataInput],6)))
print('Part 2 :', countActiveCube2(simulateNCyclePart2(convertInputToNewHyperV(dataInput),6)))

Part 1 : 372
Part 2 : 1896
